In [1]:
%%capture
%run RC4.ipynb
%run SDES.ipynb
%run Diffie-Hellman.ipynb

import socket
from _thread import start_new_thread

key = None
cipher = None
ciphers = {'rc4': RC4, 'sdes': SDES}
server = None

def listen_server():
    while server:
        try:
            show_msg(server.recv(2048))
        except:
            pass

def stop_client():
    global key
    global cipher
    global server
    key = None
    cipher = None
    if server:
        server.close()
        server = None

def start_client(ip, port):
    global server
    stop_client()
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.connect((ip, port))
    start_new_thread(listen_server, ())

def command_check(text):
    global key
    global cipher
    if text[:6] == '\crypt':
        cipher = ciphers.get(text[7:text.find(' ', 8)])
        if cipher:
            key = text[text.find(' ', 8)+1:]
        else:
            key = None
        return False
    return True

def show_msg(text):
    text = str(text, 'utf-8')
    if command_check(text) and cipher:
        print('Server :', cipher(key, text))
    else:
        print('Server :', text)

def send_msg(text):
    try:
        if command_check(text) and cipher:
            server.send(cipher(key, text).encode())
        else:
            server.send(bytes(text, 'utf-8'))
    except:
        pass

In [2]:
start_client('localhost', 5354)

Server : Hello


In [14]:
send_msg(str(diffie_hellman_public(353,3,97)))

Server : 248


In [16]:
print(diffie_hellman_key(353,97,248))

160


In [4]:
stop_client()

In [ ]:
# \crypt [sdes | rc4 | none] [key]
# tudo UTF-8

# Exemplos:
# \crypt sdes "1010101010"
# \crypt rc4 "key"
# \crypt none

# diffie_hellman_public(prime, alfa, MY_SECRET_NUMBER)
# diffie_hellman_key(prime, source_secret, destination_public_key)

# prime = 353
# alfa = 3